# Path Compression (CS170 Spoiler)

## What We've Achieved

| Implementation | Constructor | `connect` | `isConnected`|
| --- | --- | --- | --- |
|`ListOfSetsDs` | $\Theta(N)$ | $O(N)$ | $O(N)$|
| `QuickFindDs` | $\Theta(N)$ | $\Theta(N)$ | $\Theta(1)$|
|`QuickUnionDS` | $\Theta(N)$ | $O(N)$ | $O(N)$ |
| `WeightedQuickUnionDS` | $\Theta(N)$ | $O(log N)$ | $O(log N)$ |

Suppose we're about to do $M$ operations on a `DisjointSet` objects with $N$ elements.
* For our naive implementation, runtime is $O(MN)$
* For our best implementation, runtime is $O(N + M log N)$

If $N = 10^9$ and $M = 10^9$, the difference is 30 years vs. 6 seconds!
* Key point: good data structure unlocks solutions to problems that could otherwise not be solved.

What we have currently is good enough for all practical uses, but could we theoretically do better?

## CS170 Spoiler - Path Compression, A Clever Idea

Below is the topology of the worst case if we use `WeightedQuickUnion`. 

![](images/example.png)

Suppose we do `isConnected(15, 10)`. First we start with 15.
* Check 15's parent: 11
* Check 11's parent: 5
* Check 5's parent: 1
* Check 1's parent: 0
* Check 0's parent: None

We climbed the tree and spent some computation time. We'll have to do the same thing from 10 all the way to 0. 

## Clever Idea

When we did `isConnected(15, 10)`, we've found the root of 15, 11, 5 and 1, which is 0. We can now tie all these values to the root. 

![](images/tie.png)

**Very important**: This changes nothing about which set each item belongs to. 
* 15, 11, 5, and 1 are all still part of the set owned by 0. 

Recall the idea of memoization or caching. Here we're setting aside the information that 15, 11, 5, 1, 10, and 3 are all part of the set where 0 is the root.

![](images/root.png)

The additional cost of this improvement is insignificant (same order of growth), but with this implementation, future queries are faster.

## Path Compression - Another Clever Idea

Draw the tree after we call `isConnected(14, 13)`. 

![](images/draw.png)

#### Answer

![](images/draw2.png)

## CS170 Spoiler - Path Compression, A Clever Idea

Path compression results in a union/connected operations that are very close to amortized (on average) constant time.
* `M` operations on `N` nodes is $O(N + M lg^* N)$
    * $lg*$ is a function known as the iterated logarithm 
    * This means for a $lg^* N$ value, it corresponds to the $N$ of 2 to the power of the previous $N$
        * For example, $lg^* N$ of 5 corresponds to $N$ of  $2^{65536}$, where 65536 is the previous $N$
    * We'll see this in CS170
* $lg^*$ is less than 5 for any realistic input.

| $N$ | $lg* N$ |
| --- | --- |
| 1 | 0 |
| 2 | 1 |
| 4 | 2 |
| 16 | 3 |
| 65535 ($2^{16}$) | 4 |
| $2^{65536}$|5|

Turns out this is actually a somewhat pessimistic analysis. There's actually another analysis that states that the runtime won't be worse than $M lg^* N$. With a litte more cleverness, we can show that the bound is tighter.

## Path Compression - Theoretical Performance

A tighter bound: 
$$O(N + M \alpha(N))$$
...where $\alpha$ is the inverse Ackermann function.

The inverse ackermann function is less than 5 for all practical inputs
* See "Efficiency of a Good But Not Linear Set Union Algorithm"
* Written by Bob Tarjan while at UC Berkeley in 1975

## A Summary of Our Iterative Design Process

The end result of our iterative design process is the standard way disjoint sets are implemented today - quick union and path compression.

The ideas that made our implementation efficient:

* Represent sets as connected components (don't track individual connections)
    * `ListOfSetsDs` - store connected components as a List of Sets (slow, complicated)
    * `QuickFindDS` - store connected components as set ids
    * `QuickUnionDs` - store connected components as parent ids
        * `WeightedQuickUnionDS` - track the size of each set, and use `size` to decide on new tree root
            * `WeightedQuickUnionWithPathCompressionDS` - On calls to `connect` and `isConnected`, set parent `id` to the root for all items seen

## Performance Summary

| Implementation | Constructor |
| --- | --- |
|`ListOfSetsDs` | $O(NM)$ |
| `QuickFindDs` | $\Theta(NM)$ |
|`QuickUnionDS` | $O(NM)$ | 
| `WeightedQuickUnionDS` | $O(N + M log N)$ |
| `WeightedQuickUnionDSWithPathCompression` | $O(N + M \alpha(N))$|

We can think of $\alpha$ (ackermann function) as a constant since they grow very slowly.

These runtimes are given assuming:
* We have a `DisjointSets` object of size `N`
* We perform `M` operations, where an operation is defined as either a call to `connected` or `isConnected`.